In [1]:
import json
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

# Данные

## База моих чатов из телеграма
Месседжер телеграм позволяет загрузить всю переписку из всех диалогов пользователя. Такие данные довольно интересны для болталки, так как в чатах есть сохранена структура диалогов.


In [2]:
def load_telegram_results(json_fname='data/1.json'):
    with open(json_fname) as f:
        r = json.load(f)
    chats = []
    for chat in r['chats']['list']:
        if 'name' not in chat:
            chat.pop('messages')
            print("Ignore chat", chat.keys())
            print(chat)
            continue
        #print(f'***  Process chat "{chat["name"]}" ***')
        prev_id = None
        dialog = []
        seq = []
        for m in chat['messages']:
            if 'text' not in m or 'from_id' not in m:
                continue
            
            text = m['text'] #.strip()
            if isinstance(text, list):
                t_out = []
                for s in text:
                    if isinstance(s, str):
                        t_out.append(s)
                text = ' '.join(t_out).strip()
            assert isinstance(text, str), m
            
            if not text or len(text) < 3:
                continue
            if m['from_id'] != prev_id and len(seq)>0:
                if prev_id is not None:
                    dialog.append('\n'.join(seq))
                prev_id = m['from_id']
                seq = []
            
            if not isinstance(text, str):
                continue 
            seq.append(text)
        if len(dialog) > 0:
            chats.append(dialog)
    return chats
telegram_data = load_telegram_results('data/result.json')

Ignore chat dict_keys(['type', 'id'])
{'type': 'saved_messages', 'id': 109109025}


In [3]:
total_len = 0
for d in telegram_data:
    total_len+=sum(len(seq.split())for seq in d)
total_len

1292093

## База Диалогов с толоки

Датасет от Яндекс.Толока и iPavlov из 10 тыс. диалогов для обучения чат-ботов. Содержит профили с описанием личности человека и диалоги между участниками исследования. В данной работе мы будем использовать только диалоги из датасета. 



In [4]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.dialog = []
        self.last_p=None
        self.seq = []
        
    def handle_data(self, data):
        #print("Encountered some data  :", data)
        if data.startswith('Пользователь'):
            p, n, part = data.split(' ', 2)
            assert p == 'Пользователь', data
        else:
            n = self.last_p
            part = data
        if self.last_p != n and self.last_p != None:
            self.dialog.append('\n'.join(self.seq))
            self.seq = []
        self.last_p = n
        self.seq.append(part)

persona = pd.read_csv('data/TlkPersonaChatRus/dialogues.tsv', sep='\t')['dialogue']
parser = MyHTMLParser()
for p in persona:
    parser.feed(p)
parser.dialog[2]

'Что читаешь? Мне нравится классика\nЯ тоже люблю пообщаться'

In [5]:
TikPersonaChatRus = parser.dialog

In [6]:
TikPersonaChatRus[:10]

['Привет) расскажи о себе',
 'Привет) под вкусный кофеек настроение поболтать появилось\n)',
 'Что читаешь? Мне нравится классика\nЯ тоже люблю пообщаться',
 'Люблю животных, просто обожаю, как и свою работу)\nЯ фантастику люблю',
 'А я выращиваю фиалки\nИ веду здоровый и активный образ жизни!',
 'Ух ты, интересно.',
 'Ты случайно не принц на белом коне? Я его очень жду\n..',
 'А у меня из хобби каждую неделю тусить с моим лучшим\nдругом)\nПривет!',
 'Привет,Как жизнь?',
 'Отлично) Солнышко светит, птички поют!']

## База комментариев с пикабу

Датасет с коментариями с пикабу. Данный датасет содержит коментарии и метки токсичности к ним. В этом датасете нет диалогов, только вырванные из контекста комментарии. Но можно попробовать использовать его как дополнительный датасет.

In [7]:
picabu_df = pd.read_csv('data/russian_comments_from_2ch_pikabu.csv', delimiter=',')[['comment', 'toxic']]
picabu_df.dataframeName = 'russian_comments_from_2ch_pikabu.csv'
nRow, nCol = picabu_df.shape
print(f'There are {nRow} rows and {nCol} columns')
picabu_df.head()

There are 14412 rows and 2 columns


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


# Подготовка токенайзера и датасета
 

In [8]:
import sentencepiece
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruT5-base')

In [9]:
tokenizer(['Пользователь: Как дела?', 'Система: нормально'])

{'input_ids': [[25089, 23, 169, 710, 25, 2], [10909, 23, 3252, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [10]:
tokenizer('Пользователь:', add_special_tokens=False)

{'input_ids': [25089, 23], 'attention_mask': [1, 1]}

In [11]:
tokenizer('Система:',  add_special_tokens=False)

{'input_ids': [10909, 23], 'attention_mask': [1, 1]}

In [12]:
def get_substrings_items(dialogs, user_prefix_len, assistant_prefix_len, max_len=600, verbose=False):
    """Разделение реплик из диалога на реплики системы и пользователя. 
    Пользователь всегда говорит первый. 
    Так же если max_len позволяет, то добавляется история диалога
    """
    d_se = []
    for d_id, d in enumerate(dialogs):
        if verbose:
            print(f"{d_id=}, {d=}")
        for s_id in range(len(d)):
            if verbose:
                print(f"{s_id=}, {d[s_id]=}")
            curr_len=0 # len(self.up_tokenized)
            person=True
            #d_se = []
            for s_e_id in range(s_id, len(d)):
                curr_len += user_prefix_len if person else assistant_prefix_len
                curr_len += len(d[s_e_id])
                if verbose:
                    print(f"{s_e_id=}, {person=}, {curr_len=}, {d[s_e_id]=}")
                if curr_len >= max_len:
                    if verbose:
                        print(f'{curr_len=} greate than max_len.')
                    break
                if not person:
                    d_se.append((d_id, s_id, s_e_id))
                person = not person
    return d_se
                     
get_substrings_items([[[1, 2, 3], [4,5], [6,7,8,9,0], [1,2], [3], [6,7,8], [8,9,0,0,1]]], 1, 1, max_len=10, verbose=True)

d_id=0, d=[[1, 2, 3], [4, 5], [6, 7, 8, 9, 0], [1, 2], [3], [6, 7, 8], [8, 9, 0, 0, 1]]
s_id=0, d[s_id]=[1, 2, 3]
s_e_id=0, person=True, curr_len=4, d[s_e_id]=[1, 2, 3]
s_e_id=1, person=False, curr_len=7, d[s_e_id]=[4, 5]
s_e_id=2, person=True, curr_len=13, d[s_e_id]=[6, 7, 8, 9, 0]
curr_len=13 greate than max_len.
s_id=1, d[s_id]=[4, 5]
s_e_id=1, person=True, curr_len=3, d[s_e_id]=[4, 5]
s_e_id=2, person=False, curr_len=9, d[s_e_id]=[6, 7, 8, 9, 0]
s_e_id=3, person=True, curr_len=12, d[s_e_id]=[1, 2]
curr_len=12 greate than max_len.
s_id=2, d[s_id]=[6, 7, 8, 9, 0]
s_e_id=2, person=True, curr_len=6, d[s_e_id]=[6, 7, 8, 9, 0]
s_e_id=3, person=False, curr_len=9, d[s_e_id]=[1, 2]
s_e_id=4, person=True, curr_len=11, d[s_e_id]=[3]
curr_len=11 greate than max_len.
s_id=3, d[s_id]=[1, 2]
s_e_id=3, person=True, curr_len=3, d[s_e_id]=[1, 2]
s_e_id=4, person=False, curr_len=5, d[s_e_id]=[3]
s_e_id=5, person=True, curr_len=9, d[s_e_id]=[6, 7, 8]
s_e_id=6, person=False, curr_len=15, d[s_e_id]=[8, 

[(0, 0, 1), (0, 1, 2), (0, 2, 3), (0, 3, 4), (0, 4, 5)]

In [13]:
class DialogDataSet:
    """Обертка над коллекцией диалогов. 
    Выполняет добавление префиксов и токенизацию"""
    def __init__(self, tokenizer, dialogs, user_prefix='Пользователь: ', assistant_prefix='Система: ', max_len=40):
        self.dialogs = dialogs
        self.tokenizer = tokenizer
        self.user_prefix = user_prefix
        self.assistant_prefix = assistant_prefix
        self.eos_token_id = tokenizer.eos_token_id
        with tokenizer.as_target_tokenizer():
            self.up_tokenized = tokenizer(user_prefix, add_special_tokens=False)['input_ids']
            self.ap_tokenized = tokenizer(assistant_prefix, add_special_tokens=False)['input_ids']
            self.tokenized = [[s for s in tokenizer(d, add_special_tokens=False)['input_ids'] if s != [8, 0]] for d in dialogs ]
        
        self.d_s_l = get_substrings_items(self.tokenized, 
                                          len(self.up_tokenized), 
                                          len(self.ap_tokenized), 
                                          max_len=max_len)
        print(f"Dataset have {len(self.d_s_l)} examples")
    
    def get_full_seq_context(self, seq):
        return 
    
    def __getitem__(self, index):
        d, s, l = self.d_s_l[index]
        input_ids = self.up_tokenized + \
            [t for i, seq in enumerate(self.tokenized[d][s:l]) \
             for t in seq + [self.eos_token_id] + (self.ap_tokenized if i %2==0 else self.up_tokenized)]
        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(self.tokenized[d][l] + [self.eos_token_id])
        return {"input_ids":input_ids,
                "labels": labels, }
                #'attention_mask': torch.ones_like(input_ids)}
    
    def __len__(self):
        return len(self.d_s_l)
    
    def collate(self, items):
        pad_id = tokenizer.pad_token_id
        input_ids = [item['input_ids'] for item in items]
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_id)
        labels = [item['labels'] for item in items]
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pad_id)
        #attention_mask = [item['attention_mask'] for item in items]
        #torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0),
        return {"input_ids": input_ids,
                "labels": labels,
                "attention_mask": (input_ids != pad_id).float(), 
                'decoder_attention_mask': (labels != pad_id).float()}
        

In [14]:
# class PairDataSet:
#     def __init__(self, tokenizer, dialogs, max_len=40):
#         self.dialogs = dialogs
#         self.tokenizer = tokenizer
#         self.max_tokens = max_len
#         self.eos_token_id = tokenizer.eos_token_id
        
#         with tokenizer.as_target_tokenizer():
#             self.tokenized = [tokenizer(d, add_special_tokens=True, padding="max_length", truncation=True, max_length=max_len) for d in dialogs]
#         #self.input_ids = torch.cat((d['input_ids'] for d in tokenized)) 
        
#         self.d_s_l = get_substrings_items(self.tokenized, 
#                                           0, 
#                                           0, 
#                                           max_len=max_len)
#         print(f"Dataset have {len(self.d_s_l)} examples")
    
#     def __getitem__(self, index):
#         d, s, l = self.d_s_l[index]
        
#         dialog = self.tokenized[d]
                                
#         #first = 
#         #second = self.tokenized[d]
#         item = {'input_ids': dialog['input_ids'][s],
#                 'attention_mask': dialog['attention_mask'][s],
#                 'labels': dialog['input_ids'][l],
#                 'decoder_attention_mask': dialog['attention_mask'][l]}
#         return item

#     def __len__(self):
#         return len(self.d_s_l)
    


# Обучение модели

In [14]:
# Загрузка основного сета
persona_full_ds = DialogDataSet(tokenizer, [TikPersonaChatRus])
persona_small_ds = DialogDataSet(tokenizer, [TikPersonaChatRus[:1000]])
persona_tiny_ds = DialogDataSet(tokenizer, [TikPersonaChatRus[:100]], max_len=100)

Dataset have 151890 examples
Dataset have 1074 examples
Dataset have 256 examples


In [16]:
# train_full = DialogDataSet(tokenizer, [TikPersonaChatRus, *telegram_data])

In [17]:
print(tokenizer.convert_ids_to_tokens(persona_tiny_ds[16]['input_ids']))
print(tokenizer.convert_ids_to_tokens(persona_tiny_ds[16]['labels']))

['▁Пользователь', ':', '▁', '▁Ты', '▁случайно', '▁не', '▁принц', '▁на', '▁белом', '▁коне', '?', '▁Я', '▁его', '▁очень', '▁жду', '▁', '.', '.', '</s>', '▁Система', ':', '▁']
['▁А', '▁у', '▁меня', '▁из', '▁хобби', '▁каждую', '▁неделю', '▁', 'тус', 'ить', '▁с', '▁моим', '▁лучшим', '▁другом', ')', '▁Привет', '!', '</s>']


## Обучение на persona_small_ds

In [68]:
model = AutoModelForSeq2SeqLM.from_pretrained('sberbank-ai/ruT5-base')
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"t5-sber-finetuned-small",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10, 
    do_eval=False,
    logging_steps = 10,
logging_strategy='steps')

loading configuration file https://huggingface.co/sberbank-ai/ruT5-base/resolve/main/config.json from cache at /home/mitrofanov-aa/.cache/huggingface/transformers/15f5f9138b337892ef8dadfab622952847d6eb4b5985a825847fbaa18538bae9.d9b947fcfcea30df5eb71effa3afe0a2e3da3535463b97a14c3c0401ac680b99
Model config T5Config {
  "_name_or_path": "/home/jovyan/models/t5/t5_base_org",
  "_num_labels": 2,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0

In [69]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=persona_small_ds,
    data_collator=persona_small_ds.collate,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [70]:
trainer.train()

***** Running training *****
  Num examples = 1074
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 680


Step,Training Loss
10,9.525200
20,6.825100
30,4.729400
40,3.742400
50,3.351500
60,3.128500
70,3.187500
80,2.477100
90,2.556400
100,2.463000


Saving model checkpoint to t5-sber-finetuned-small/checkpoint-500
Configuration saved in t5-sber-finetuned-small/checkpoint-500/config.json
Model weights saved in t5-sber-finetuned-small/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-small/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-small/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-small/checkpoint-500/spiece.model


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=680, training_loss=2.1644571023828845, metrics={'train_runtime': 139.0833, 'train_samples_per_second': 77.22, 'train_steps_per_second': 4.889, 'total_flos': 459875008235520.0, 'train_loss': 2.1644571023828845, 'epoch': 10.0})

In [71]:
torch.save(model, f'models/2e_persona_small.pt')

In [74]:
def dict_to_device(d, device):
    return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k,v in d.items()}

with torch.no_grad():
    batch = persona_small_ds.collate([persona_small_ds[2]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=2, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))
    
        
    #print(out_ids)
    
        
    

Пользователь:  Что читаешь? Мне нравится классика Я тоже люблю пообщаться
 Система: 
Лейбл >>>  Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю

***System variants***
Вариант 0: >>>  В чем ты любишь путешествовать?
Вариант 1: >>>  Привет
Вариант 2: >>>  Привет Привет) А как дела? Как твои дела?
Вариант 3: >>>  Привет! А ты?
Вариант 4: >>>  А ты? Какими чувствами увлекаешься?
Вариант 5: >>>  У меня тоже дети есть, а у тебя?
Вариант 6: >>>  а у меня есть собака
Вариант 7: >>>  Я тоже люблю готовить
Вариант 8: >>>  Да, я тоже люблю море
Вариант 9: >>>  а ты?


In [25]:
def interractive(model, tokenizer, user_prefix='Пользователь: ', assistant_prefix='Система: ', context_len=40, num_lines=10):
    context=''
    input_line = input('<<< ')
    with torch.no_grad():
        for i in range(num_lines):
            if not input_line:
                break
            context+=user_prefix + input_line + ' </s> ' + assistant_prefix.strip()
            #print(context)
            t = tokenizer(context, add_special_tokens=False)
            #print(t)
            #print(tokenizer.convert_ids_to_tokens(t['input_ids']))
            input_ids = t['input_ids'][-context_len:]
            input_ids=torch.LongTensor(t['input_ids']).cuda().view(1,-1)
            attention_mask = torch.ones_like(input_ids).cuda()
            out_ids = model.generate(input_ids=input_ids, 
                           attention_mask=attention_mask,
                           num_beams=2, do_sample=True)
            s = tokenizer.decode(out_ids.cpu().squeeze(), skip_special_tokens=True)
            print('>>> ', s)
            context+=s.strip() + ' </s> '
            
            input_line = input('<<< ')

In [75]:
interractive(model, tokenizer)

<<<  Привет! Как дела?


>>>  Я живу в Москве, а ты?


<<<  я в Питере. Учусь в ИТМО. А ты чем занимаешься? 


>>>  Я живу в Омске


<<<  Круто там?


...

<<<  Я тоже


>>>  Как дела?


<<<  хорошо


>>>  Привет


<<<  ужас. Хватит здороваться 


## Обучение на persona_full_ds

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained('sberbank-ai/ruT5-base')
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"t5-sber-finetuned-small",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, 
    do_eval=False,
    logging_steps = 100,
logging_strategy='steps')

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=persona_full_ds,
    data_collator=persona_full_ds.collate,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 151890
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18988


Step,Training Loss
100,4.110100
200,2.158200
300,2.001100
400,1.939900
500,1.895300
600,1.834500
700,1.893100
800,1.838300
900,1.830800
1000,1.794900


Saving model checkpoint to t5-sber-finetuned-small/checkpoint-500
Configuration saved in t5-sber-finetuned-small/checkpoint-500/config.json
Model weights saved in t5-sber-finetuned-small/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-small/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-small/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-small/checkpoint-500/spiece.model
Saving model checkpoint to t5-sber-finetuned-small/checkpoint-1000
Configuration saved in t5-sber-finetuned-small/checkpoint-1000/config.json
Model weights saved in t5-sber-finetuned-small/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-small/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-small/checkpoint-1000/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-small/checkpoint-1000/spiece.model
Saving model checkpoint to t5-sber-finetuned

TrainOutput(global_step=18988, training_loss=1.6301456191852668, metrics={'train_runtime': 3504.8035, 'train_samples_per_second': 86.675, 'train_steps_per_second': 5.418, 'total_flos': 1.256067468711936e+16, 'train_loss': 1.6301456191852668, 'epoch': 2.0})

In [21]:
#torch.save(model, f'models/2e_persona_full.pt')
model = torch.load(f'models/2e_persona_full.pt')

In [33]:
with torch.no_grad():
    batch = persona_small_ds.collate([persona_small_ds[2]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=4, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))

Пользователь:  Что читаешь? Мне нравится классика Я тоже люблю пообщаться
 Система: 
Лейбл >>>  Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю

***System variants***
Вариант 0: >>>  А я люблю читать. А ты чем занимаешься в свободное время?
Вариант 1: >>>  Я люблю читать детективы
Вариант 2: >>>  Я люблю классику, а ты?
Вариант 3: >>>  Я люблю читать книги, а ты что читаешь?
Вариант 4: >>>  Я тоже люблю читать, особенно классику. А ты читаешь?
Вариант 5: >>>  А ты любишь читать?
Вариант 6: >>>  Я люблю шоппинг, а ты?
Вариант 7: >>>  Привет! Как дела? Чем занимаешься?
Вариант 8: >>>  А я люблю музыку А ты?
Вариант 9: >>>  У тебя есть семья?


In [36]:
interractive(model, tokenizer)

<<<  Привет! Я Антон, студент второго курса. Люблю программирование, а ты? 


>>>  Я уже на пенсии, поэтому не работаю. Работаю в интернете.


<<<  Ничего себе ты продвинутый дед =)


>>>  Какую музыку любишь?


<<<  Да всякую. Но рок больше всего. А ты слушаешь рок?


...

<<<  неважно. Сколько будет два плюс два?


>>>  Как тебя зовут?


<<<  Антон Митрофанов


>>>  Чем занимаешься?


<<<  студент


In [80]:
model.cpu()
pass

## Обучение на telegram

In [17]:
telegram_full = DialogDataSet(tokenizer, telegram_data)

Dataset have 90680 examples


In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained('sberbank-ai/ruT5-base')
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"t5-sber-finetuned-tel",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, 
    do_eval=False,
    logging_steps = 1000,
logging_strategy='steps')

loading configuration file https://huggingface.co/sberbank-ai/ruT5-base/resolve/main/config.json from cache at /home/mitrofanov-aa/.cache/huggingface/transformers/15f5f9138b337892ef8dadfab622952847d6eb4b5985a825847fbaa18538bae9.d9b947fcfcea30df5eb71effa3afe0a2e3da3535463b97a14c3c0401ac680b99
Model config T5Config {
  "_name_or_path": "/home/jovyan/models/t5/t5_base_org",
  "_num_labels": 2,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0

In [19]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=telegram_full,
    data_collator=telegram_full.collate,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 90680
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11336


Step,Training Loss
1000,2.593800
2000,2.251400
3000,2.199300
4000,2.179000
5000,2.158600
6000,2.156000
7000,2.119700
8000,2.128900
9000,2.088000
10000,2.107900


Saving model checkpoint to t5-sber-finetuned-tel/checkpoint-500
Configuration saved in t5-sber-finetuned-tel/checkpoint-500/config.json
Model weights saved in t5-sber-finetuned-tel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-tel/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-tel/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-tel/checkpoint-500/spiece.model
Deleting older checkpoint [t5-sber-finetuned-tel/checkpoint-4500] due to args.save_total_limit
Saving model checkpoint to t5-sber-finetuned-tel/checkpoint-1000
Configuration saved in t5-sber-finetuned-tel/checkpoint-1000/config.json
Model weights saved in t5-sber-finetuned-tel/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-tel/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-tel/checkpoint-1000/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-tel/ch

TrainOutput(global_step=11336, training_loss=2.1866430095492237, metrics={'train_runtime': 2156.7547, 'train_samples_per_second': 84.089, 'train_steps_per_second': 5.256, 'total_flos': 7414756910161920.0, 'train_loss': 2.1866430095492237, 'epoch': 2.0})

In [31]:
#torch.save(model, f'models/2e_telegram_full.pt')
model = torch.load(f'models/2e_telegram_full.pt')

In [22]:
with torch.no_grad():
    batch = persona_small_ds.collate([persona_small_ds[1]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=2, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))

Пользователь:  Привет) под вкусный кофеек настроение поболтать появилось )
 Система: 
Лейбл >>>  Что читаешь? Мне нравится классика Я тоже люблю пообщаться

***System variants***
Вариант 0: >>>  ахах
Вариант 1: >>>  спасибо!


/mnt/hdd/mitrofanov-aa/project/study/anaconda/envs/sr/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Вариант 2: >>>  го го?
Вариант 3: >>>  Ооооооооооооооооооо
Вариант 4: >>>  Спасибо)
Вариант 5: >>>  Привет!
Вариант 6: >>>  Привет, )
Вариант 7: >>>  У меня тоже
Вариант 8: >>>  а что вы там делаете?
Вариант 9: >>>  Привет) Не забудь)


In [23]:
with torch.no_grad():
    batch = telegram_full.collate([telegram_full[1]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=2, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))

Пользователь:  ты кидал бумажку для участия в лотерее?
 Система: 
Лейбл >>>  что за бумажку? куда ее надо было кидать?

***System variants***
Вариант 0: >>>  я не попал в эту лотерею
Вариант 1: >>>  , я думал, что ты сдал
Вариант 2: >>>  Я кидал, но не кидал
Вариант 3: >>>  Я её не кидал, она была в конверте
Вариант 4: >>>  Я кидал бумажку в папку с чатом
Вариант 5: >>>  я кинул её в лотерею
Вариант 6: >>>  Ну да, я кидал)
Вариант 7: >>>  мб она не нужна?
Вариант 8: >>>  Ну это не так важно)
Вариант 9: >>>  А, ну я взял её и поменял, но мне не понравилось, я хз,


In [38]:
interractive(model, tokenizer, context_len=80, num_lines=5)

KeyboardInterrupt: Interrupted by user

## Обучение на telegram и persona_full_ds

In [44]:
pt_full = DialogDataSet(tokenizer, [TikPersonaChatRus, *telegram_data])

Dataset have 242570 examples


In [45]:
model = AutoModelForSeq2SeqLM.from_pretrained('sberbank-ai/ruT5-base')
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"t5-sber-finetuned-per_tel",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, 
    do_eval=False,
    logging_steps = 10,
logging_strategy='steps')

loading configuration file https://huggingface.co/sberbank-ai/ruT5-base/resolve/main/config.json from cache at /home/mitrofanov-aa/.cache/huggingface/transformers/15f5f9138b337892ef8dadfab622952847d6eb4b5985a825847fbaa18538bae9.d9b947fcfcea30df5eb71effa3afe0a2e3da3535463b97a14c3c0401ac680b99
Model config T5Config {
  "_name_or_path": "/home/jovyan/models/t5/t5_base_org",
  "_num_labels": 2,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0

In [46]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=pt_full,
    data_collator=pt_full.collate,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 242570
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 30322


Step,Training Loss
10,9.151300
20,6.901700
30,5.050000
40,4.447500
50,3.830800
60,3.569300
70,3.141900
80,2.876300
90,2.569700
100,2.666900


Saving model checkpoint to t5-sber-finetuned-per_tel/checkpoint-500
Configuration saved in t5-sber-finetuned-per_tel/checkpoint-500/config.json
Model weights saved in t5-sber-finetuned-per_tel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-per_tel/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-per_tel/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-per_tel/checkpoint-500/spiece.model
Saving model checkpoint to t5-sber-finetuned-per_tel/checkpoint-1000
Configuration saved in t5-sber-finetuned-per_tel/checkpoint-1000/config.json
Model weights saved in t5-sber-finetuned-per_tel/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-per_tel/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-per_tel/checkpoint-1000/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-per_tel/checkpoint-1000/spiece.model
Saving model checkpo

In [58]:
#torch.save(model, f'models/2e_person_tel_full.pt')
model = torch.load(f'models/2e_person_tel_full.pt')

In [64]:
with torch.no_grad():
    batch = persona_small_ds.collate([persona_small_ds[1]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=2, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))

Пользователь:  Привет) под вкусный кофеек настроение поболтать появилось )
 Система: 
Лейбл >>>  Что читаешь? Мне нравится классика Я тоже люблю пообщаться

***System variants***
Вариант 0: >>>  Привет) Как дела?
Вариант 1: >>>  Привет)
Вариант 2: >>>  Привет! Как тебя зовут?
Вариант 3: >>>  А ты чем занимаешься?
Вариант 4: >>>  Привет!
Вариант 5: >>>  Я работаю юристом. А ты чем занимаешься?
Вариант 6: >>>  Привет
Вариант 7: >>>  а я люблю пиццу и пельмени
Вариант 8: >>>  Привет, как дела?
Вариант 9: >>>  Привет, я работаю продавцом, а ты?


In [60]:
with torch.no_grad():
    batch = telegram_full.collate([telegram_full[1]])
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    print('='*40)
    print("***System variants***")
    for i in range(10):
        out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                                 attention_mask=batch['attention_mask'].cuda(),
                                 num_beams=2, do_sample=True)
        print(f"Вариант {i}: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))

Пользователь:  ты кидал бумажку для участия в лотерее?
 Система: 
Лейбл >>>  что за бумажку? куда ее надо было кидать?

***System variants***
Вариант 0: >>>  Я кидал, но не видел
Вариант 1: >>>  Да
Вариант 2: >>>  Нет, щас буду с ней играть
Вариант 3: >>>  нет
Вариант 4: >>>  Нет
Вариант 5: >>>  я кидал
Вариант 6: >>>  нет
Вариант 7: >>>  Ну да
Вариант 8: >>>  Я так и делаю, но у меня ее нет
Вариант 9: >>>  неа


In [71]:
interractive(model, tokenizer, context_len=80, num_lines=5)

<<<  Привет! Давай познакомимся? меня Антон зовут


>>>  Меня зовут Алина, мне 25, а тебе сколько лет?


<<<  Вау, мне тоже 25


>>>  Чем занимаешься?


<<<  Учусь. Пытаюсь вот с помощью тебя сдать проект


...

<<<  Программистом. А ты? 


>>>  Учусь на программиста


<<<  Могу помочь в этом деле)


>>>  Я работаю в отделе кадров


<<<  Нанимаешь работников? Большая компания?
